In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-core==0.2.9 --progress-bar off
!pip -qqq install langchain-community==0.2.5 --progress-bar off
!pip -qqq install langchain-groq==0.1.5 --progress-bar off
!pip -qqq install crewai[tools]==0.32.0 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.17 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.2.9 which is incompatible.
langchain-text-splitters 0.2.4 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.2.9 which is incompatible.


1 - This Agent is responsible to change several uploaded csv files into sqlLite.

2 - After Uploading and as a first step, using a for loop to give the table columns "SHOW COLUMNS FROM TABLE[i];".

3 - Inside the for loop and after displaying the column names in each table, store the results in a variable and store them to feed them to master "input to an LLM".

In [ ]:
# initialize DB agent functionalities, nothing has to do with LLms at the moment
# will start working on this until I have received a result from the Web Scraper from Nour's side

In [30]:
import pandas as pd

df = pd.read_csv('csv_test/testtest.csv')
df1 = pd.read_csv('csv_test/testtest1.csv')


In [3]:
import sqlite3
conn = sqlite3.connect('testtest.db') # create a database called testtest.db


df.to_sql('testtest', conn, if_exists='replace', index=False)
# inserting testtest.csv into the database as testtest


122

In [4]:
# inserting testtest1.csv into the database as testtest1
df1.to_sql('testtest1', conn, if_exists='replace', index=False)

50985

In [5]:
my_set = set()
my_set.add('testtest')
my_set.add('testtest1')
my_set

{'testtest', 'testtest1'}

We transformed our csv files into tables in our "testtest" database which can be queried

In [6]:
# Query the database to fetch all rows from the 'testtest' table
result = pd.read_sql_query("SELECT * FROM testtest", conn)
result.head()


,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.0


In [7]:
result1 = pd.read_sql_query("SELECT * FROM testtest1", conn)
result1.head()

,Year,Industry_aggregation_NZSIOC,Industry_code_NZSIOC,Industry_name_NZSIOC,Units,Variable_code,Variable_name,Variable_category,Value,Industry_code_ANZSIC06
0,2023,Level 1,99999,All industries,Dollars (millions),H01,Total income,Financial performance,930995,ANZSIC06 divisions A-S (excluding classes K633...
1,2023,Level 1,99999,All industries,Dollars (millions),H04,"Sales, government funding, grants and subsidies",Financial performance,821630,ANZSIC06 divisions A-S (excluding classes K633...
2,2023,Level 1,99999,All industries,Dollars (millions),H05,"Interest, dividends and donations",Financial performance,84354,ANZSIC06 divisions A-S (excluding classes K633...
3,2023,Level 1,99999,All industries,Dollars (millions),H07,Non-operating income,Financial performance,25010,ANZSIC06 divisions A-S (excluding classes K633...
4,2023,Level 1,99999,All industries,Dollars (millions),H08,Total expenditure,Financial performance,832964,ANZSIC06 divisions A-S (excluding classes K633...


We will next extract the meta data from our dataset and append them into a "table_columns"

In [11]:
column_names = pd.read_sql_query("PRAGMA table_info(testtest)", conn)
column_names

,cid,name,type,notnull,dflt_value,pk
0,0,customerNumber,INTEGER,0,None,0
1,1,customerName,TEXT,0,None,0
2,2,contactLastName,TEXT,0,None,0
3,3,contactFirstName,TEXT,0,None,0
4,4,phone,TEXT,0,None,0
5,5,addressLine1,TEXT,0,None,0
6,6,addressLine2,TEXT,0,None,0
7,7,city,TEXT,0,None,0
8,8,state,TEXT,0,None,0
9,9,postalCode,TEXT,0,None,0


In [ ]:
table_columns = ''
for i in range(0, len(my_set)): 
    table_name = list(my_set)[i] 
    column_info = pd.read_sql_query(f"PRAGMA table_info({table_name})", conn)
    column_names = column_info['name'].tolist()
    table_columns += 'Table '+ table_name + ':\n'  +'Has Columns: '+ ','.join(column_names) + ','# Join column names with commas
    # new line
    table_columns += '\n\n'
print(table_columns)

Table testtest:
Has Columns: customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,

Table testtest1:
Has Columns: Year,Industry_aggregation_NZSIOC,Industry_code_NZSIOC,Industry_name_NZSIOC,Units,Variable_code,Variable_name,Variable_category,Value,Industry_code_ANZSIC06,




We have successfully stored the tables and their column names and we are now ready to send them to the main agent to analyze and then send the request to the web scraper

In [15]:
column_names1 = pd.read_sql_query("PRAGMA table_info(testtest1)", conn)
column_names1

,cid,name,type,notnull,dflt_value,pk
0,0,Year,INTEGER,0,None,0
1,1,Industry_aggregation_NZSIOC,TEXT,0,None,0
2,2,Industry_code_NZSIOC,TEXT,0,None,0
3,3,Industry_name_NZSIOC,TEXT,0,None,0
4,4,Units,TEXT,0,None,0
5,5,Variable_code,TEXT,0,None,0
6,6,Variable_name,TEXT,0,None,0
7,7,Variable_category,TEXT,0,None,0
8,8,Value,TEXT,0,None,0
9,9,Industry_code_ANZSIC06,TEXT,0,None,0
